# Description

This architecture was described in "Deep learning with convolutional neural networks for brain mapping and decoding of movement-related information from the human EEG", by R. T. Schirrmeister et al, 2018. In this notebook we conduct experiments showing dependency between accuracy and the number of timestamps in a sample. 

# Set up the environment

In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

# import tf
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.backend as K

# import os functions
import os
import time

import numpy as np
import matplotlib.pyplot as plt

from IPython import display

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
cd 'drive/My Drive/147 Project'

/content/drive/My Drive/147 Project


# Load the data

### Read the files

In [0]:
X_test = np.load("./EEG_data/X_test.npy")
y_test = np.load("./EEG_data/y_test.npy") - 769
person_train_valid = np.load("./EEG_data/person_train_valid.npy")
X_train_valid = np.load("./EEG_data/X_train_valid.npy")
y_train_valid = np.load("./EEG_data/y_train_valid.npy") - 769
person_test = np.load("./EEG_data/person_test.npy")

### Shape of data

In [8]:
print("training/Valid data shape: {}".format(X_train_valid.shape))       # training data of many persons
print("Test data shape: {}".format(X_test.shape))                        # test data of many persons
print("Training/Valid target shape: {}".format(y_train_valid.shape))     # training labels of many persons
print("Test target shape: {}".format(y_test.shape))                      # test labels of many persons
print("Person train/valid  shape: {}".format(person_train_valid.shape))  # which person correspond to the trail in test set
print("Person test shape: {}".format(person_test.shape))                 # which person correspond to the trail in test set

training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid  shape: (2115, 1)
Person test shape: (443, 1)


### divide dataset into training and validation

In [9]:
perm = np.random.permutation(X_train_valid.shape[0])
num_train = int(0.8 * X_train_valid.shape[0])
num_valid = X_train_valid.shape[0] - num_train
X_train =  X_train_valid[perm[0:num_train]]
y_train =  y_train_valid[perm[0:num_train]]
X_valid = X_train_valid[perm[num_train: ]]
y_valid = y_train_valid[perm[num_train: ]]


print("Training data shape: {}".format(X_train.shape))
print("Training label shape: {}".format(y_train.shape))
print("Validation data shape: {}".format(X_valid.shape))
print("Validation label shape: {}".format(y_valid.shape))
print("Test data shape: {}".format(X_test.shape))
print("Test label shape: {}".format(y_test.shape))

Training data shape: (1692, 22, 1000)
Training label shape: (1692,)
Validation data shape: (423, 22, 1000)
Validation label shape: (423,)
Test data shape: (443, 22, 1000)
Test label shape: (443,)


### Preprocess data

In [0]:
def sliding_window(X_arr, y_arr, time_window=100, time_step=1, time_stride=1):
    temp_x = np.moveaxis(X_arr, 2, 0)
    temp_x = temp_x.astype(np.float32)
    buff = []
    
    num_slices = (len(temp_x)-time_window*time_step) // time_stride + 1
    
    # get time slices for data
    for i in range(num_slices):
        buff.append(temp_x[i*time_stride:i*time_stride + time_window*time_step:time_step])
        buff[i] = np.moveaxis(buff[i], 0, 2)
        # uncomment this if additional dimension is needed
        # buff[i] = buff[i].reshape(1, buff[i].shape[0], buff[i].shape[1], buff[i].shape[2])
        
    temp_x = np.concatenate(buff)
        
    # get time slice for labels
    temp_y = np.ones((X_arr.shape[0],num_slices))
    
    for i in range(len(y_arr)):
        temp_y[i] = temp_y[i] * y_arr[i]
        
    temp_y = temp_y.reshape((-1))
    
    return temp_x, temp_y

In [0]:
def Ksquare(x):
    return K.pow(x, 2)

def Klog(x):
    return K.log(x)

In [0]:
def construct_shallow_model(TIME_WINDOW):
    # input
    shallow_input = layers.Input(shape=(22, TIME_WINDOW))

    # conv accross time domain
    r1 = layers.Reshape((22, TIME_WINDOW, 1))(shallow_input)
    c1 = layers.Conv2D(40, (1, 25), strides=(1, 1), activation="elu")(r1)
    new_size = TIME_WINDOW - 25 + 1
    t1 = tf.keras.layers.Permute((2, 3, 1))(c1)
    
    
    # conv accross time domain
    r2 = layers.Reshape((new_size, 40*22, 1))(t1)
    c2 = layers.Conv2D(40, (1, 40*22), strides=(1, 1), activation="elu")(r2)

    sq1 = layers.Activation(Ksquare)(c2)
    r3 = layers.Reshape((new_size, 40, 1))(sq1)
    apool1 = layers.AveragePooling2D(pool_size=(75, 1), strides=(15, 1))(r3)

    log1 = layers.Activation(Klog)(apool1)
    f1 = layers.Flatten()(log1)

    # output
    shallow_output = layers.Dense(4, activation="softmax")(f1)
    
    return keras.Model(inputs = shallow_input, outputs = shallow_output)

### Train on single person

In [16]:
person_num = 0
indices_train_valid = np.where(person_train_valid == person_num)[0]
indices_test = np.where(person_test == person_num)[0]

single_person_X_train_valid = X_train_valid[indices_train_valid]
single_person_y_train_valid = y_train_valid[indices_train_valid]

perm = np.random.permutation(single_person_X_train_valid.shape[0])
num_train = int(0.8 * single_person_X_train_valid.shape[0])
num_valid = single_person_X_train_valid.shape[0] - num_train
single_person_X_train =  single_person_X_train_valid[perm[0:num_train]]
single_person_y_train =  single_person_y_train_valid[perm[0:num_train]]
single_person_X_valid = single_person_X_train_valid[perm[num_train: ]]
single_person_y_valid = single_person_y_train_valid[perm[num_train: ]]

single_person_X_test = X_test[indices_test]
single_person_y_test = y_test[indices_test]


print("Training data shape for 1 person: {}".format(single_person_X_train.shape))
print("Training label shape for 1 person: {}".format(single_person_y_train.shape))
print("Validation data shape for 1 person: {}".format(single_person_X_valid.shape))
print("Validation label shape for 1 person: {}".format(single_person_y_valid.shape))
print("Test data shape for 1 person: {}".format(single_person_X_test.shape))
print("Test label shape for 1 person: {}".format(single_person_y_test.shape))

TIME_WINDOW = 600
TIME_STRIDE = 1000

# cut the slices
X_train_slices, y_train_slices = sliding_window(single_person_X_train, 
                                                single_person_y_train, 
                                                time_window=TIME_WINDOW,  
                                                time_stride=TIME_STRIDE)


X_valid_slices, y_valid_slices = sliding_window(single_person_X_valid, 
                                                single_person_y_valid, 
                                                time_window=TIME_WINDOW, 
                                                time_stride=TIME_STRIDE)

shallow_model_600 = construct_shallow_model(TIME_WINDOW)
shallow_model_600.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

shallow_model_600.fit(X_train_slices, y_train_slices,
                      validation_data = (X_valid_slices, y_valid_slices),
                      epochs = 30)

X_test_slices, y_test_slices = sliding_window(single_person_X_test, 
                                              single_person_y_test, 
                                              time_window=600, 
                                              time_stride=TIME_STRIDE)


X_overall_test_slices, y_overall_test_slices = sliding_window(X_test, 
                                              y_test, 
                                              time_window=600, 
                                              time_stride=TIME_STRIDE)

print("\n\n Now Evaluating on Single Person test set")

shallow_model_600.evaluate(X_test_slices, y_test_slices)

print("\n\n Now Evaluating on Entire test set")

shallow_model_600.evaluate(X_overall_test_slices, y_overall_test_slices)

Training data shape for 1 person: (189, 22, 1000)
Training label shape for 1 person: (189,)
Validation data shape for 1 person: (48, 22, 1000)
Validation label shape for 1 person: (48,)
Test data shape for 1 person: (50, 22, 1000)
Test label shape for 1 person: (50,)
Train on 189 samples, validate on 48 samples
Epoch 1/30
189/189 [==============================] - 4s 22ms/sample - loss: 2.6653 - acc: 0.2698 - val_loss: 2.5440 - val_acc: 0.2292
Epoch 2/30
189/189 [==============================] - 4s 21ms/sample - loss: 1.9872 - acc: 0.2593 - val_loss: 2.1237 - val_acc: 0.1250
Epoch 3/30
189/189 [==============================] - 4s 21ms/sample - loss: 1.4062 - acc: 0.4286 - val_loss: 1.9442 - val_acc: 0.2500
Epoch 4/30
189/189 [==============================] - 4s 21ms/sample - loss: 1.1599 - acc: 0.4550 - val_loss: 1.8049 - val_acc: 0.3333
Epoch 5/30
189/189 [==============================] - 4s 21ms/sample - loss: 1.0859 - acc: 0.5291 - val_loss: 1.7095 - val_acc: 0.3542
Epoch 6/30
1

[2.3170282054970013, 0.3589165]

In [17]:
person_num = 1
indices_train_valid = np.where(person_train_valid == person_num)[0]
indices_test = np.where(person_test == person_num)[0]

single_person_X_train_valid = X_train_valid[indices_train_valid]
single_person_y_train_valid = y_train_valid[indices_train_valid]

perm = np.random.permutation(single_person_X_train_valid.shape[0])
num_train = int(0.8 * single_person_X_train_valid.shape[0])
num_valid = single_person_X_train_valid.shape[0] - num_train
single_person_X_train =  single_person_X_train_valid[perm[0:num_train]]
single_person_y_train =  single_person_y_train_valid[perm[0:num_train]]
single_person_X_valid = single_person_X_train_valid[perm[num_train: ]]
single_person_y_valid = single_person_y_train_valid[perm[num_train: ]]

single_person_X_test = X_test[indices_test]
single_person_y_test = y_test[indices_test]


print("Training data shape for 1 person: {}".format(single_person_X_train.shape))
print("Training label shape for 1 person: {}".format(single_person_y_train.shape))
print("Validation data shape for 1 person: {}".format(single_person_X_valid.shape))
print("Validation label shape for 1 person: {}".format(single_person_y_valid.shape))
print("Test data shape for 1 person: {}".format(single_person_X_test.shape))
print("Test label shape for 1 person: {}".format(single_person_y_test.shape))

TIME_WINDOW = 600
TIME_STRIDE = 1000

# cut the slices
X_train_slices, y_train_slices = sliding_window(single_person_X_train, 
                                                single_person_y_train, 
                                                time_window=TIME_WINDOW,  
                                                time_stride=TIME_STRIDE)


X_valid_slices, y_valid_slices = sliding_window(single_person_X_valid, 
                                                single_person_y_valid, 
                                                time_window=TIME_WINDOW, 
                                                time_stride=TIME_STRIDE)

shallow_model_600 = construct_shallow_model(TIME_WINDOW)
shallow_model_600.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

shallow_model_600.fit(X_train_slices, y_train_slices,
                      validation_data = (X_valid_slices, y_valid_slices),
                      epochs = 30)

X_test_slices, y_test_slices = sliding_window(single_person_X_test, 
                                              single_person_y_test, 
                                              time_window=600, 
                                              time_stride=TIME_STRIDE)


X_overall_test_slices, y_overall_test_slices = sliding_window(X_test, 
                                              y_test, 
                                              time_window=600, 
                                              time_stride=TIME_STRIDE)

print("\n\n Now Evaluating on Single Person test set")

shallow_model_600.evaluate(X_test_slices, y_test_slices)

print("\n\n Now Evaluating on Entire test set")

shallow_model_600.evaluate(X_overall_test_slices, y_overall_test_slices)

Training data shape for 1 person: (188, 22, 1000)
Training label shape for 1 person: (188,)
Validation data shape for 1 person: (48, 22, 1000)
Validation label shape for 1 person: (48,)
Test data shape for 1 person: (50, 22, 1000)
Test label shape for 1 person: (50,)
Train on 188 samples, validate on 48 samples
Epoch 1/30
188/188 [==============================] - 4s 22ms/sample - loss: 3.1876 - acc: 0.2500 - val_loss: 1.9402 - val_acc: 0.2708
Epoch 2/30
188/188 [==============================] - 4s 22ms/sample - loss: 1.7671 - acc: 0.3191 - val_loss: 1.9768 - val_acc: 0.2708
Epoch 3/30
188/188 [==============================] - 4s 22ms/sample - loss: 1.3972 - acc: 0.4362 - val_loss: 1.6209 - val_acc: 0.3125
Epoch 4/30
188/188 [==============================] - 4s 22ms/sample - loss: 1.2491 - acc: 0.4840 - val_loss: 1.5473 - val_acc: 0.3333
Epoch 5/30
188/188 [==============================] - 4s 22ms/sample - loss: 1.1656 - acc: 0.4947 - val_loss: 1.5730 - val_acc: 0.2500
Epoch 6/30
1

[1.8202736488045204, 0.3227991]

In [18]:
person_num = 2
indices_train_valid = np.where(person_train_valid == person_num)[0]
indices_test = np.where(person_test == person_num)[0]

single_person_X_train_valid = X_train_valid[indices_train_valid]
single_person_y_train_valid = y_train_valid[indices_train_valid]

perm = np.random.permutation(single_person_X_train_valid.shape[0])
num_train = int(0.8 * single_person_X_train_valid.shape[0])
num_valid = single_person_X_train_valid.shape[0] - num_train
single_person_X_train =  single_person_X_train_valid[perm[0:num_train]]
single_person_y_train =  single_person_y_train_valid[perm[0:num_train]]
single_person_X_valid = single_person_X_train_valid[perm[num_train: ]]
single_person_y_valid = single_person_y_train_valid[perm[num_train: ]]

single_person_X_test = X_test[indices_test]
single_person_y_test = y_test[indices_test]


print("Training data shape for 1 person: {}".format(single_person_X_train.shape))
print("Training label shape for 1 person: {}".format(single_person_y_train.shape))
print("Validation data shape for 1 person: {}".format(single_person_X_valid.shape))
print("Validation label shape for 1 person: {}".format(single_person_y_valid.shape))
print("Test data shape for 1 person: {}".format(single_person_X_test.shape))
print("Test label shape for 1 person: {}".format(single_person_y_test.shape))

TIME_WINDOW = 600
TIME_STRIDE = 1000

# cut the slices
X_train_slices, y_train_slices = sliding_window(single_person_X_train, 
                                                single_person_y_train, 
                                                time_window=TIME_WINDOW,  
                                                time_stride=TIME_STRIDE)


X_valid_slices, y_valid_slices = sliding_window(single_person_X_valid, 
                                                single_person_y_valid, 
                                                time_window=TIME_WINDOW, 
                                                time_stride=TIME_STRIDE)

shallow_model_600 = construct_shallow_model(TIME_WINDOW)
shallow_model_600.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

shallow_model_600.fit(X_train_slices, y_train_slices,
                      validation_data = (X_valid_slices, y_valid_slices),
                      epochs = 30)

X_test_slices, y_test_slices = sliding_window(single_person_X_test, 
                                              single_person_y_test, 
                                              time_window=600, 
                                              time_stride=TIME_STRIDE)


X_overall_test_slices, y_overall_test_slices = sliding_window(X_test, 
                                              y_test, 
                                              time_window=600, 
                                              time_stride=TIME_STRIDE)

print("\n\n Now Evaluating on Single Person test set")

shallow_model_600.evaluate(X_test_slices, y_test_slices)

print("\n\n Now Evaluating on Entire test set")

shallow_model_600.evaluate(X_overall_test_slices, y_overall_test_slices)

Training data shape for 1 person: (188, 22, 1000)
Training label shape for 1 person: (188,)
Validation data shape for 1 person: (48, 22, 1000)
Validation label shape for 1 person: (48,)
Test data shape for 1 person: (50, 22, 1000)
Test label shape for 1 person: (50,)
Train on 188 samples, validate on 48 samples
Epoch 1/30
188/188 [==============================] - 4s 21ms/sample - loss: 3.1978 - acc: 0.2234 - val_loss: 1.8917 - val_acc: 0.2292
Epoch 2/30
188/188 [==============================] - 4s 21ms/sample - loss: 1.8667 - acc: 0.2766 - val_loss: 1.4935 - val_acc: 0.3125
Epoch 3/30
188/188 [==============================] - 4s 21ms/sample - loss: 1.4128 - acc: 0.3564 - val_loss: 1.5395 - val_acc: 0.3333
Epoch 4/30
188/188 [==============================] - 4s 21ms/sample - loss: 1.2233 - acc: 0.4415 - val_loss: 1.5067 - val_acc: 0.3542
Epoch 5/30
188/188 [==============================] - 4s 21ms/sample - loss: 1.1520 - acc: 0.5426 - val_loss: 1.4002 - val_acc: 0.3750
Epoch 6/30
1

[2.0410146304231733, 0.38148984]

In [19]:
person_num = 3
indices_train_valid = np.where(person_train_valid == person_num)[0]
indices_test = np.where(person_test == person_num)[0]

single_person_X_train_valid = X_train_valid[indices_train_valid]
single_person_y_train_valid = y_train_valid[indices_train_valid]

perm = np.random.permutation(single_person_X_train_valid.shape[0])
num_train = int(0.8 * single_person_X_train_valid.shape[0])
num_valid = single_person_X_train_valid.shape[0] - num_train
single_person_X_train =  single_person_X_train_valid[perm[0:num_train]]
single_person_y_train =  single_person_y_train_valid[perm[0:num_train]]
single_person_X_valid = single_person_X_train_valid[perm[num_train: ]]
single_person_y_valid = single_person_y_train_valid[perm[num_train: ]]

single_person_X_test = X_test[indices_test]
single_person_y_test = y_test[indices_test]


print("Training data shape for 1 person: {}".format(single_person_X_train.shape))
print("Training label shape for 1 person: {}".format(single_person_y_train.shape))
print("Validation data shape for 1 person: {}".format(single_person_X_valid.shape))
print("Validation label shape for 1 person: {}".format(single_person_y_valid.shape))
print("Test data shape for 1 person: {}".format(single_person_X_test.shape))
print("Test label shape for 1 person: {}".format(single_person_y_test.shape))

TIME_WINDOW = 600
TIME_STRIDE = 1000

# cut the slices
X_train_slices, y_train_slices = sliding_window(single_person_X_train, 
                                                single_person_y_train, 
                                                time_window=TIME_WINDOW,  
                                                time_stride=TIME_STRIDE)


X_valid_slices, y_valid_slices = sliding_window(single_person_X_valid, 
                                                single_person_y_valid, 
                                                time_window=TIME_WINDOW, 
                                                time_stride=TIME_STRIDE)

shallow_model_600 = construct_shallow_model(TIME_WINDOW)
shallow_model_600.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

shallow_model_600.fit(X_train_slices, y_train_slices,
                      validation_data = (X_valid_slices, y_valid_slices),
                      epochs = 30)

X_test_slices, y_test_slices = sliding_window(single_person_X_test, 
                                              single_person_y_test, 
                                              time_window=600, 
                                              time_stride=TIME_STRIDE)


X_overall_test_slices, y_overall_test_slices = sliding_window(X_test, 
                                              y_test, 
                                              time_window=600, 
                                              time_stride=TIME_STRIDE)

print("\n\n Now Evaluating on Single Person test set")

shallow_model_600.evaluate(X_test_slices, y_test_slices)

print("\n\n Now Evaluating on Entire test set")

shallow_model_600.evaluate(X_overall_test_slices, y_overall_test_slices)

Training data shape for 1 person: (187, 22, 1000)
Training label shape for 1 person: (187,)
Validation data shape for 1 person: (47, 22, 1000)
Validation label shape for 1 person: (47,)
Test data shape for 1 person: (50, 22, 1000)
Test label shape for 1 person: (50,)
Train on 187 samples, validate on 47 samples
Epoch 1/30
187/187 [==============================] - 4s 22ms/sample - loss: 3.0955 - acc: 0.2620 - val_loss: 2.3464 - val_acc: 0.1489
Epoch 2/30
187/187 [==============================] - 4s 21ms/sample - loss: 1.8707 - acc: 0.2995 - val_loss: 1.6337 - val_acc: 0.2766
Epoch 3/30
187/187 [==============================] - 4s 21ms/sample - loss: 1.4164 - acc: 0.3850 - val_loss: 1.9562 - val_acc: 0.2340
Epoch 4/30
187/187 [==============================] - 4s 21ms/sample - loss: 1.2065 - acc: 0.4920 - val_loss: 1.5173 - val_acc: 0.3617
Epoch 5/30
187/187 [==============================] - 4s 21ms/sample - loss: 1.1413 - acc: 0.5348 - val_loss: 1.5082 - val_acc: 0.3617
Epoch 6/30
1

[2.2270621973558686, 0.36343116]